In [ ]:
#The following code adapted from
#https://medium.com/@RareLoot/extracting-spotify-data-on-your-favourite-artist-via-python-d58bc92a4330

# To collect data on Justin Bieber (modify as needed)
name = "Justin Bieber" #chosen artist
result = sp.search(name) #search query
result['tracks']['items'][0]['artists']

#Extract Artist's uri
artist_uri = result['tracks']['items'][0]['artists'][0]['uri']
#Pull all of the artist's albums
sp_albums = sp.artist_albums(artist_uri, album_type='album')
#Store artist's albums' names' and uris in separate lists
album_names = []
album_uris = []
for i in range(len(sp_albums['items'])):
    album_names.append(sp_albums['items'][i]['name'])
    album_uris.append(sp_albums['items'][i]['uri'])
    
album_names
#album_uris
#Keep names and uris in same order to keep track of duplicate albums

def albumSongs(uri):
    album = uri #assign album uri to a_name
    spotify_albums[album] = {} #Creates dictionary for that specific album
    #Create keys-values of empty lists inside nested dictionary for album
    spotify_albums[album]['album'] = [] #create empty list
    spotify_albums[album]['track_number'] = []
    spotify_albums[album]['id'] = []
    spotify_albums[album]['name'] = []
    spotify_albums[album]['uri'] = []
    tracks = sp.album_tracks(album) #pull data on album tracks
    for n in range(len(tracks['items'])): #for each song track
        spotify_albums[album]['album'].append(album_names[album_count])
        spotify_albums[album]['track_number'].append(tracks['items'][n]['track_number'])
        spotify_albums[album]['id'].append(tracks['items'][n]['id'])
        spotify_albums[album]['name'].append(tracks['items'][n]['name'])
        spotify_albums[album]['uri'].append(tracks['items'][n]['uri'])

spotify_albums = {}
album_count = 0
for i in album_uris: #each album
    albumSongs(i)
    print("Album " + str(album_names[album_count]) + " songs has been added to spotify_albums dictionary")
    album_count+=1 #Updates album count once all tracks have been added

def audio_features(album):
    #Add new key-values to store audio features
    spotify_albums[album]['Danceability'] = []
    spotify_albums[album]['Energy'] = []
    spotify_albums[album]['Speechness'] = []
    spotify_albums[album]['Acousticness'] = []
    spotify_albums[album]['Instrumentalness'] = []
    spotify_albums[album]['Liveness'] = []
    spotify_albums[album]['Valence'] = []

    #create a track counter
    track_count = 0
    for track in spotify_albums[album]['uri']:
        #pull audio features per track
        features = sp.audio_features(track)
        #Append to relevant key-value
        spotify_albums[album]['Danceability'].append(features[0]['danceability'])
        spotify_albums[album]['Energy'].append(features[0]['energy'])
        spotify_albums[album]['Speechness'].append(features[0]['speechiness'])
        spotify_albums[album]['Acousticness'].append(features[0]['acousticness'])
        spotify_albums[album]['Instrumentalness'].append(features[0]['instrumentalness'])
        spotify_albums[album]['Liveness'].append(features[0]['liveness'])
        spotify_albums[album]['Valence'].append(features[0]['valence'])

import time
import numpy as np
sleep_min = 2
sleep_max = 5
start_time = time.time()
request_count = 0
for i in spotify_albums:
    audio_features(i)
    request_count+=1
    if request_count % 5 == 0:
        print(str(request_count) + " playlists completed")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        print('Loop #: {}'.format(request_count))
        print('Elapsed Time: {} seconds'.format(time.time() - start_time))

dic_df = {}
dic_df['album'] = []
dic_df['track_number'] = []
dic_df['id'] = []
dic_df['name'] = []
dic_df['uri'] = []
dic_df['Acousticness'] = []
dic_df['Danceability'] = []
dic_df['Energy'] = []
dic_df['Instrumentalness'] = []
dic_df['Liveness'] = []
dic_df['Speechness'] = []
dic_df['Valence'] = []
for album in spotify_albums: 
    for feature in spotify_albums[album]:
        if feature == 'acousticness':
            feature2 = 'Acousticness'
        elif feature == 'danceability':
            feature2 = 'Danceability'
        elif feature == 'energy':
            feature2 = 'Engergy'
        elif feature == 'instrumentalness':
            feature2 = 'Instrumentalness'
        elif feature == 'liveness':
            feature2 = 'Liveness'
        elif feature == 'speechiness':
            feature2 = 'Speechness'
        elif feature == 'valence':
            feature2 = 'Valence'
        else:
            feature2 = feature
        dic_df[feature2].extend(spotify_albums[album][feature])
        
len(dic_df['album'])

import pandas as pd
df = pd.DataFrame.from_dict(dic_df)
df

dic_vectors = {}
counter = 0
#print(df[name])
for track in df['name']:
    dic_vectors[track] = [df['Danceability'][counter], df['Energy'][counter], df['Speechness'][counter], df['Acousticness'][counter], df['Instrumentalness'][counter], df['Liveness'][counter], df['Valence'][counter]]
#    print(track)
    counter += 1

print(dic_vectors)

final_bieber = pd.DataFrame.from_dict(dic_vectors)
final_bieber.to_csv(r'final_bieber_data.csv')
